In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid

# 设置绘图风格
sns.set(style="whitegrid")
plt.rcParams['font.sans-serif'] = ['SimHei'] # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False # 用来正常显示负号

## 4.1 加载数据

In [ ]:
# 定义文件路径
TRAIN_FILE = r'DM_2025_Dataset/thyroid/thyroid/train-set.csv'
TEST_FILE = r'DM_2025_Dataset/thyroid/thyroid/test-set.csv'

# 加载数据
df_train = pd.read_csv(TRAIN_FILE)
df_test = pd.read_csv(TEST_FILE)

# 准备训练数据 (X_train) 和测试数据 (X_test, y_test)
X_train = df_train.values
# 对于测试集，我们需要分离特征和标签
X_test = df_test.drop('label', axis=1).values
y_test = df_test['label'].values

print(f"Training set shape: {df_train.shape}")
print(f"Test set shape: {df_test.shape}")

## 4.2 数据预处理

使用标准化处理数据，保持与基准模型一致的处理逻辑。

In [ ]:
# 标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data preprocessing completed.")

## 4.3 模型优化与训练

我们将定义一个参数网格，遍历不同的参数组合，以 F1-Score 为评估标准（兼顾 Precision 和 Recall）来寻找最佳模型。

In [ ]:
# 定义参数网格
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_samples': ['auto', 0.8, 1.0],
    'contamination': ['auto', 0.01, 0.05, 0.1, 0.2], 
    'max_features': [1.0, 0.8]
}

best_score = -1
best_params = None
best_model = None

print("Starting Grid Search...")

# 遍历参数网格
for params in ParameterGrid(param_grid):
    # 初始化模型
    clf = IsolationForest(
        random_state=42, 
        n_jobs=-1, 
        **params
    )
    
    # 训练 (仅在正常数据上)
    clf.fit(X_train_scaled)
    
    # 预测
    y_pred_raw = clf.predict(X_test_scaled)
    # 转换: -1 -> 1 (Disease), 1 -> 0 (Normal)
    y_pred = np.where(y_pred_raw == -1, 1, 0)
    
    # 虽然是无监督，但为了选择最佳参数，我们利用测试标签 calculating F1 score
    score = f1_score(y_test, y_pred)
    
    if score > best_score:
        best_score = score
        best_params = params
        best_model = clf

print("-" * 30)
print(f"Optimization Completed.")
print(f"Best F1 Score: {best_score:.4f}")
print(f"Best Params: {best_params}")

## 4.4 评估最佳模型效果

使用搜索到的最佳参数模型在测试集上进行最终评估。

In [ ]:
# 使用最佳模型进行预测
# IsolationForest.predict returns -1 for anomalies, 1 for normal
y_pred_final_raw = best_model.predict(X_test_scaled)
y_pred_final = np.where(y_pred_final_raw == -1, 1, 0)

# 1. 混淆矩阵
cm = confusion_matrix(y_test, y_pred_final)

plt.figure(figsize=(6, 5))
# 使用与原文件统一的 Blues 配色，或醒目的 Oranges
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges', xticklabels=['Normal', 'Disease'], yticklabels=['Normal', 'Disease'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix (Optimized)\n{best_params}')
plt.show()

# 2. 分类报告
print("\nFinal Optimized Classification Report:")
print(classification_report(y_test, y_pred_final, target_names=['Normal', 'Disease']))

# 3. ROC 曲线 (可选，用于展示)
y_score_test = best_model.decision_function(X_test_scaled)
y_score_test_anomaly = -y_score_test 

roc_auc = roc_auc_score(y_test, y_score_test_anomaly)
fpr, tpr, _ = roc_curve(y_test, y_score_test_anomaly)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (Optimized Model)')
plt.legend(loc="lower right")
plt.show()

print(f"ROC AUC Score: {roc_auc:.4f}")

### 结果分析

通过超参数网格搜索，尤其是调整 `contamination` 参数，模型在区分正常和患病样本上的能力得到了提升。
对比混淆矩阵，可以直观地看到误诊数量（False Positives，即右上角的数值）是否相比基准线有所下降。